https://esdac.jrc.ec.europa.eu/content/lucas2015-topsoil-data

In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} openpyxl folium matplotlib mapclassify

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [4]:
# LUCAS TOPSOIL 2009 datasheets
lts_09 = r"D:\soil_data\LUCAS_TOPSOIL_v1\LUCAS_TOPSOIL_v1.xlsx"
lts_15 = r"D:\soil_data\LUCAS2015_topsoildata_20200323\LUCAS_Topsoil_2015_20200323.csv"
lts_DE = r'D:\soil_data\LUCAS2015_topsoildata_20200323_micro\LUCAS_Topsoil_2015_DE.csv'

In [5]:
# LUCAS TOPSOIL 2009
# Load excel file with sample_ID as index.
df_09 = pd.read_excel(lts_09)
# Unselect rows with invalid POINT_ID.
df_09 = df_09[df_09['POINT_ID'].astype(str).str.isdigit()]
# Set POINT_ID as index.
df_09.set_index("POINT_ID", inplace=True)
# Rename to consider equality between 2009 and 2015 data
df_09.rename(index={'POINT_ID': 'Point_ID'}, inplace=True) 
# Add columns from 2015 survey 
df_09[['Elevation','LC1', 'LU1', 'Soil_Stones', "NUTS_0","NUTS_1","NUTS_2","NUTS_3", "LC1_Desc","LU1_Desc"]] = np.nan

In [6]:
# LUCAS TOPSOIL 2015
df_15 = pd.read_csv(lts_15, index_col="Point_ID")
# Update missing landcover information from the 2015 survey
df_09.update(df_15)
#Reset Index 
df_09.reset_index(inplace=True)

In [7]:
# German Points Ides
df_DE = pd.read_csv(lts_DE, index_col="Point_ID")
DE_Points = list(df_DE.index)

In [8]:
# Creating a GeoDataFrame from a DataFrame with coordinates
df_09 = gpd.GeoDataFrame(df_09, geometry=gpd.points_from_xy(df_09.GPS_LONG, df_09.GPS_LAT), crs=4326)
# Load area of interest into geopandas for masking df_09
aoi = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\oberrheingraben_shp\oberrheingraben-polygon.shp').to_crs(epsg=4326)
# Subset data
df_grassland = df_09[df_09.LC1_Desc == 'Grassland without tree/shrub cover']
df_grassland_DE = df_grassland[df_grassland['POINT_ID'].isin(DE_Points)]
# Save data to disk 
df_grassland.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_grassland.csv')
df_grassland_DE.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_grassland_DE.csv')

In [7]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
greece = world.loc[world['name'] == 'Germany']

In [9]:
from irrigation_detection import get_s1_s2_era5_df
from glob import glob

In [10]:
longitude = list(df_grassland_DE.geometry.x)
latitude = list(df_grassland_DE.geometry.y)
point_id = list(df_grassland_DE.POINT_ID)

In [11]:
filepath_grassland = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland/'
filepath_grassland_DE = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/'
filepath_bare_soil = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/bare_soil/'
files = glob(filepath_grassland_DE + '*')
existing_ids = [int(x.split('_')[7]) for x in files]

In [ ]:
for lon, lat, idx in zip(longitude, latitude, point_id):
    if idx not in existing_ids:
        get_s1_s2_era5_df(
            longitudes = [lon],
            latitudes = [lat],
            polygon = None,
            ids = [idx], 
            filepath = filepath_grassland_DE,
            start = '2017-01-01', 
            end = '2020-01-01',
            scale_s1 = 30,
            scale_s2 = 30,
            scale_era5 = 30,
            idx_name = 'POINT_ID',
            get_grid_scale = False
        )
    else:
        print(f'{idx} data already exists... skip download!')

46643144 data already exists... skip download!
46563124 data already exists... skip download!
46323122 data already exists... skip download!
46083304 data already exists... skip download!
46043206 data already exists... skip download!
46023376 data already exists... skip download!
46003390 data already exists... skip download!
45863256 data already exists... skip download!
45863206 data already exists... skip download!
45823098 data already exists... skip download!
45743216 data already exists... skip download!
45703230 data already exists... skip download!
45702852 data already exists... skip download!
45663298 data already exists... skip download!
45662866 data already exists... skip download!
45623286 data already exists... skip download!
45623070 data already exists... skip download!
45562852 data already exists... skip download!
45543160 data already exists... skip download!
45463192 data already exists... skip download!
45462850 data already exists... skip download!
45383306 data

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43682776_10.63067_48.09577.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 10.60212, latitude: 47.95194, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43662760_10.60212_47.95194.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 10.6017, latitude: 47.91596, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43662756_10.6017_

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43502746_10.3871_47.82694.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 10.37251, latitude: 49.48223, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43482930_10.37251_49.48223.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 10.36067, latitude: 47.86301, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43482750_10.36067

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43262834_10.06778_48.61937.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 10.0133, latitude: 47.68361, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43222730_10.0133_47.68361.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 9.98449, latitude: 54.62402, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43203502_9.98449_54

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43042730_9.77372_47.68338.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 9.70712, latitude: 54.46196, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43023484_9.70712_54.46196.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 9.74476, latitude: 48.15125, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_43022782_9.74476_48.

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/POINT_ID_42763074_9.36203_50.77584.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 9.29831, latitude: 51.24289, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!


https://keras.io/examples/timeseries/timeseries_classification_from_scratch/

In [ ]:
array(['Shrubland without tree cover', nan,
       'Shrubland with sparse tree cover', 'Olive groves',
       'Pine dominated coniferous woodland',
       'Pine dominated mixed woodland',
       'Spontaneously re-vegetated surfaces', 'Sunflower',
       'Other root crops', 'Common wheat', 'Broadleaved woodland',
       'Cotton', 'Other mixed woodland', 'Durum wheat', 'Oranges',
       'Clovers', 'Lucerne', 'Maize', 'Inland fresh running water',
       'Tobacco', 'Other fruit trees and berries', 'Soya',
       'Other coniferous woodland', 'Nuts trees', 'Vineyards',
       'OTHER BARE SOIL', 'Other non-permanent industrial crops',
       'Tomatoes', 'Grassland without tree/shrub cover', 'Oats',
       'Mix of cereals', 'Grassland with sparse tree/shrub cover', 'Rye',
       'Barley', 'Rape and turnip rape', 'Dry pulses',
       'Spruce dominated coniferous woodland',
       'Spruce dominated mixed woodland',
       'Other Leguminous  and mixtures for fodder', 'Cherry fruit',
       'Other fresh vegetables', 'Temporary grassland', 'Triticale',
       'Apple fruit', 'Potatoes', 'Nurseries', 'Sugar beet',
       'Non built-up linear features', 'OTHER ARTIFICIAL AREAS',
       'Other cereals', 'LICHENS AND MOSS', 'SAND', 'Peatbogs',
       'Strawberries', 'Inland marshes', 'Pear fruit',
       'Other fibre and oleaginous crops', 'Other citrus fruit',
       'Permanent industrial crops', 'Rice',
       'Floriculture and ornamental plants', 'ROCKS AND STONES',
       'Inland salty water bodies', 'Inland fresh water bodies',
       'Salines', 'Non built-up area features'], dtype=object)

In [ ]:
#Subset points from Hessisches Ried
p_id = [42142960, 42122956, 42142970, 42082950]
df_p = df_09[df_09.POINT_ID.isin(p_id)]

In [ ]:
#Export Data
#df_09_subset.to_csv(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_subset.csv')
#df_09_subset.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.shp')
#df_09_subset.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.kml')
#df_p.to_csv(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_p_ried.csv')